In [1]:
from keras.optimizers import Adam
from keras.layers import Input, Dense, LSTM, Conv1D, Dropout, Bidirectional, Multiply
from keras.models import Model
from keras.layers import Concatenate
from keras.layers.core import *
from keras.layers import LSTM
from keras.models import *
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn import metrics
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
#from keras.models import Sequential
from numpy.random import seed
from utils import *
from model import *

In [8]:
import random
fix_seed = 2021
random.seed(fix_seed)
#torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [3]:
dataframe = pd.read_csv(r'F:\ML_Bootcamp\capstone_project\dataset\test.csv')

In [5]:
dataframe.head()

,date,Open,High,Low,Close,Volume
0,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,1.856566e+10
1,2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,2.080208e+10
2,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,2.811148e+10
3,2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,1.844427e+10
4,2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,1.972507e+10


In [6]:
dataframe.isna().sum()

date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [9]:
train_size = int(len(dataframe)*.8)

In [10]:
train_df = dataframe[:train_size]
test_df = dataframe[train_size:]
test_df.reset_index(drop=True,inplace=True)

In [11]:
train_df.tail()

,date,Open,High,Low,Close,Volume
781,2022-02-20,40118.101563,40119.890625,38112.812500,38431.378906,1.834058e+10
782,2022-02-21,38423.210938,39394.437500,36950.476563,37075.281250,2.928040e+10
783,2022-02-22,37068.769531,38359.855469,36488.933594,38286.027344,2.549315e+10
784,2022-02-23,38285.281250,39122.394531,37201.816406,37296.570313,2.184907e+10
785,2022-02-24,37278.566406,38968.839844,34459.218750,38332.609375,4.638380e+10


In [12]:
test_df.head()

,date,Open,High,Low,Close,Volume
0,2022-02-25,38333.746094,39630.324219,38111.343750,39214.218750,2.654560e+10
1,2022-02-26,39213.082031,40005.347656,38702.535156,39105.148438,1.746755e+10
2,2022-02-27,39098.699219,39778.941406,37268.976563,37709.785156,2.345013e+10
3,2022-02-28,37706.000000,43760.457031,37518.214844,43193.234375,3.569001e+10
4,2022-03-01,43194.503906,44793.601563,42952.585938,44354.636719,3.247905e+10


In [13]:
scaled_train_df = pd.DataFrame(train_df['date'])

scaler_open = MinMaxScaler(feature_range=(0, 1))
scaler_high = MinMaxScaler(feature_range=(0, 1))
scaler_low = MinMaxScaler(feature_range=(0, 1))
scaler_close = MinMaxScaler(feature_range=(0, 1))
scaler_volume = MinMaxScaler(feature_range=(0, 1))

o = np.array(train_df['Open'])
h = np.array(train_df['High'])
l = np.array(train_df['Low'])
c = np.array(train_df['Close'])
v = np.array(train_df['Volume'])

scaler_open.fit(o.reshape(-1,1))
scaler_high.fit(h.reshape(-1,1))
scaler_low.fit(l.reshape(-1,1))
scaler_close.fit(c.reshape(-1,1))
scaler_volume.fit(v.reshape(-1,1))

scaled_train_df['Open'] = scaler_open.transform(o.reshape(-1,1))
scaled_train_df['High'] = scaler_high.transform(h.reshape(-1,1))
scaled_train_df['Low'] = scaler_low.transform(l.reshape(-1,1))
scaled_train_df['Close'] = scaler_close.transform(c.reshape(-1,1))
scaled_train_df['Volume'] = scaler_volume.transform(v.reshape(-1,1))

In [14]:
scaled_test_df = pd.DataFrame(test_df['date'])

o = np.array(test_df['Open'])
h = np.array(test_df['High'])
l = np.array(test_df['Low'])
c = np.array(test_df['Close'])
v = np.array(test_df['Volume'])

scaled_test_df['Open'] = scaler_open.transform(o.reshape(-1,1))
scaled_test_df['High'] = scaler_high.transform(h.reshape(-1,1))
scaled_test_df['Low'] = scaler_low.transform(l.reshape(-1,1))
scaled_test_df['Close'] = scaler_close.transform(c.reshape(-1,1))
scaled_test_df['Volume'] = scaler_volume.transform(v.reshape(-1,1))